In [1]:
import pickle
from pathlib import Path

In [7]:
p = set(pickle.loads(Path('./performance/output/valid_ids.pkl').read_bytes()))
r = set(pickle.loads(Path('./raf/output/valid_ids.pkl').read_bytes()))

In [8]:
len(p), len(r)

(6234, 15287)

In [9]:
len(p-r), len(r-p)

(990, 10043)

In [10]:
c = p|r
len(set(c))

16277

In [12]:
sorted(r-p)[:10]

[0, 14, 29, 33, 53, 75, 76, 86, 90, 94]

In [14]:
680 in p

False